# DF + LIB

In [16]:
# df that have:
# user as numeric feature (from 0 to 5) 
%store -r merge_unique_numain_df

In [17]:
# package that we need
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.metrics import MeanAbsolutePercentageError
from tensorflow.keras.regularizers import l2

#for the CROSS-VALIDATION
from sklearn.model_selection import KFold

# essential
import numpy as np
import matplotlib.pyplot as plt

# not necessary in this case
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error

# CROSS VALIDATION 

we want to try this approach in order to optimize our recomandation system, more in detail our NN

# GENARALIZE CRSS VALIDATION - all users

In [18]:
# TARGET_var
target='valence'

#FEATURES
features=[    'spotify_track_duration_minute', 'danceability', 'energy', 'key', 'loudness',
    'mode', 'speechiness', 'acousticness', 'tempo', 'time_signature',
    'spotify_track_popularity', 'instrumentalness', 'liveness']

## DEFINE FUNCTIONS

In [19]:
# function to create the model
def create_model_user(shape_x_train): #number of columns -> shape
    
    
    '''model_user_i = Sequential([
        Dense(32, input_dim=shape_x_train, activation='relu', name='first_layer'),
        # 64 as the number of initial layer is a common choice when we build a NN --> VEDIAMO COME FUNZIA POI NEL CASO SI SISTEMA IN SEGUITO
        # input_dim: take the dimension of the x_train array --> that is equivalent to the number of feature in input
        
        # SI POSSONO AGGIUNGERE PER OGNI LAYER --> Dropout(0.3) per evitare overfitting
        

        Dense(16, activation='relu', name='secod_layer'),
        Dense(1, activation='linear', name='final_layer')
    ])'''

    model_user_i = Sequential([
        
        Dense(32,input_dim=shape_x_train,activation='relu',name='first_layer',kernel_regularizer=l2(0.01)),
        # 64 as the number of initial layer is a common choice when we build a NN --> VEDIAMO COME FUNZIA POI NEL CASO SI SISTEMA IN SEGUITO
        # input_dim: take the dimension of the x_train array --> that is equivalent to the number of feature in input
        
        # SI POSSONO AGGIUNGERE PER OGNI LAYER --> Dropout(0.3) per evitare overfitting
        Dropout(0.3),
        Dense(16,activation='relu',name='secod_layer',kernel_regularizer=l2(0.01)),
        Dropout(0.2),
        #Dense(1,activation='linear',name='final_layer')
        Dense(1,activation='sigmoid',name='final_layer') 
        #ATENZION PROVIAMO SIGMOID PRCHè CI SERVE UN OUTPUT NEL RANGE DI O E 1
    
    ])

    #compile the model
    model_user_i.compile(optimizer = 'adam', loss = 'mse', metrics = ['mae'])

    #summary
    #display(model_user_i.summary())

    #return  the model
    return model_user_i

#function to train the model
def fit_model(model, x_train, y_train, x_test, y_test):

    history_model = model.fit(
        x_train,y_train,
        #validation_split=0.2,
        validation_data = [x_test, y_test],
        epochs=100,
        batch_size=32,
        #verbose=1
    )

    return history_model

def plot_for_model(history, y_hat, x_test):

    #plot loss
    plt.figure(figsize=(10,6))
    plt.plot(history.history['loss'],color='blue',label='loss')
    plt.plot(history.history['val_loss'],color='red',label='val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss function through epoch')
    plt.legend()
    plt.grid()
    plt.show()

    #plot mean
    plt.figure(figsize=(10,6))
    plt.plot(history.history['mae'],color='blue',label='mae')
    plt.plot(history.history['val_mae'],color='red',label='val_mae')
    plt.xlabel('Epoch')
    plt.ylabel('mae')
    plt.title('Mae function through epoch')
    plt.grid()
    plt.legend()
    plt.show()   

def return_train_test_for_cross(train_index, test_index, df):
    #extract the feature and target
    x = df[features]
    y = df[target]

    #normalization
    scaler = StandardScaler()
    x[features] = scaler.fit_transform(x[features])
    #display(x)

    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    #display(x_train)

    return x_train,x_test,y_train,y_test

## MAIN

In [20]:
dizio_model_user = {} #in this dizio we store the model for each user
dizio_accuracy = {}

# DICTIONARY TO STORE SONG RECOMMENDATION PER USER
reccomandations_songs_per_user={}

#value of folder
k = 5

#loop creation user and model
for i in range(0, len(merge_unique_numain_df['User'].unique())): #i indica lo user   
    
    print('---------------------------------------------------------')
    print(f'USER {i}')
    
    #select user
    mask_user = merge_unique_numain_df['User'] == i
    #display(mask_user)
    user_i_df = merge_unique_numain_df[mask_user]
    #display(user_i_df) 

    kf = KFold(n_splits = k, shuffle= True, random_state= 42)
    score_fold = []

    loop = 0
    for train_index, test_index in kf.split(user_i_df):

        print('---------------------------------------------------------')
        print(f'K {loop} FOLD') #print(f'K {} FOLD = test set')
        loop = loop + 1

        #return train and test
        x_train,x_test,y_train,y_test = return_train_test_for_cross(train_index, test_index, user_i_df) #da modificare

        #creation model for the specific user
        model_user_i = create_model_user(x_train.shape[1])
        
        #fit the model with the different x, y train (for different user)
        history_model_user_i = fit_model(model_user_i, x_train, y_train, x_test, y_test)

        #y_hat = model_user_i.predict(x_test)

        test_loss, test_mae = model_user_i.evaluate(x_test,y_test,verbose=1)
        score_fold.append(test_mae)

        
    dizio_model_user[f"User{i}"] = {#'model_user_i' : model_user_i,
                                    #'history_model_user_i': history_model_user_i,
                                    #'x_train': x_train,
                                    #'x_test': x_test,
                                    #'y_train': y_train,
                                    #'y_test': y_test,
                                    #'y_hat': y_hat,
                                    #'recommendations:': reccomandation_df
                                    'score_fold': score_fold
                                    } 
    #save accuracy 
    dizio_accuracy[f'User{i}'] = {
        #'MAE' : mean_absolute_error(y_test, y_hat),
        #'MSE' : mean_squared_error(y_test, y_hat)
        'Mean MAE': np.mean(score_fold)
    }

---------------------------------------------------------
USER 0
---------------------------------------------------------
K 0 FOLD


C:\Users\elped\AppData\Local\Temp\ipykernel_13956\4226922763.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[features] = scaler.fit_transform(x[features])
c:\Users\elped\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.3765 - mae: 0.2001 - val_loss: 0.1739 - val_mae: 0.1652
Epoch 2/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1484 - mae: 0.1765 - val_loss: 0.0778 - val_mae: 0.1596
Epoch 3/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0723 - mae: 0.1667 - val_loss: 0.0490 - val_mae: 0.1547
Epoch 4/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0502 - mae: 0.1631 - val_loss: 0.0414 - val_mae: 0.1506
Epoch 5/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0451 - mae: 0.1616 - val_loss: 0.0393 - val_mae: 0.1503
Epoch 6/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0422 - mae: 0.1565 - val_loss: 0.0384 - val_mae: 0.1480
Epoch 7/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0409 - mae: 0.1540 - val_loss: 0.0380 - val_mae: 0.1473
Epoch 8/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0412 - mae: 0.1542 - val_loss: 0.0378 - val_mae: 0.1469
Epoch 9/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/

C:\Users\elped\AppData\Local\Temp\ipykernel_13956\4226922763.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[features] = scaler.fit_transform(x[features])
c:\Users\elped\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


143/143 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.3608 - mae: 0.1916 - val_loss: 0.1712 - val_mae: 0.1748
Epoch 2/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1418 - mae: 0.1746 - val_loss: 0.0792 - val_mae: 0.1693
Epoch 3/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0700 - mae: 0.1675 - val_loss: 0.0513 - val_mae: 0.1620
Epoch 4/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0487 - mae: 0.1607 - val_loss: 0.0439 - val_mae: 0.1596
Epoch 5/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0442 - mae: 0.1595 - val_loss: 0.0416 - val_mae: 0.1558
Epoch 6/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0420 - mae: 0.1568 - val_loss: 0.0410 - val_mae: 0.1527
Epoch 7/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0399 - mae: 0.1526 - val_loss: 0.0402 - val_mae: 0.1514
Epoch 8/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0406 - mae: 0.1536 - val_loss: 0.0400 - val_mae: 0.1534
Epoch 9/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss:

...

63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 0.4370 - mae: 0.2099 - val_loss: 0.3102 - val_mae: 0.1816
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2864 - mae: 0.1894 - val_loss: 0.2070 - val_mae: 0.1757
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1938 - mae: 0.1843 - val_loss: 0.1406 - val_mae: 0.1705
Epoch 4/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1333 - mae: 0.1797 - val_loss: 0.0984 - val_mae: 0.1640
Epoch 5/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0959 - mae: 0.1737 - val_loss: 0.0727 - val_mae: 0.1594
Epoch 6/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0728 - mae: 0.1675 - val_loss: 0.0577 - val_mae: 0.1552
Epoch 7/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0578 - mae: 0.1603 - val_loss: 0.0495 - val_mae: 0.1527
Epoch 8/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0517 - mae: 0.1613 - val_loss: 0.0447 - val_mae: 0.1504
Epoch 9/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0466 - mae: 0.1

C:\Users\elped\AppData\Local\Temp\ipykernel_13956\4226922763.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[features] = scaler.fit_transform(x[features])
c:\Users\elped\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.4377 - mae: 0.2217 - val_loss: 0.3188 - val_mae: 0.2011
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2914 - mae: 0.2005 - val_loss: 0.2133 - val_mae: 0.1898
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1954 - mae: 0.1896 - val_loss: 0.1450 - val_mae: 0.1815
Epoch 4/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1316 - mae: 0.1744 - val_loss: 0.1019 - val_mae: 0.1737
Epoch 5/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0945 - mae: 0.1703 - val_loss: 0.0749 - val_mae: 0.1648
Epoch 6/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0704 - mae: 0.1624 - val_loss: 0.0593 - val_mae: 0.1591
Epoch 7/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0584 - mae: 0.1603 - val_loss: 0.0504 - val_mae: 0.1555
Epoch 8/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0519 - mae: 0.1617 - val_loss: 0.0454 - val_mae: 0.1530
Epoch 9/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0450 - mae: 0.1

C:\Users\elped\AppData\Local\Temp\ipykernel_13956\4226922763.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[features] = scaler.fit_transform(x[features])
c:\Users\elped\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.4349 - mae: 0.2041 - val_loss: 0.3075 - val_mae: 0.1854
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2807 - mae: 0.1863 - val_loss: 0.2010 - val_mae: 0.1796
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1853 - mae: 0.1807 - val_loss: 0.1334 - val_mae: 0.1724
Epoch 4/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1241 - mae: 0.1722 - val_loss: 0.0924 - val_mae: 0.1670
Epoch 5/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0902 - mae: 0.1723 - val_loss: 0.0688 - val_mae: 0.1636
Epoch 6/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0702 - mae: 0.1704 - val_loss: 0.0551 - val_mae: 0.1601
Epoch 7/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0554 - mae: 0.1612 - val_loss: 0.0471 - val_mae: 0.1563
Epoch 8/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0502 - mae: 0.1587 - val_loss: 0.0432 - val_mae: 0.1524
Epoch 9/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0461 - mae: 0.1

# DISPLAY DICTIONARY ACCURAY

In [21]:
display(dizio_accuracy)

{'User0': {'Mean MAE': 0.14134439527988435},
 'User1': {'Mean MAE': 0.13658384680747987},
 'User2': {'Mean MAE': 0.10007672011852264},
 'User3': {'Mean MAE': 0.1349403440952301},
 'User4': {'Mean MAE': 0.1395527243614197}}